In [1]:
from __future__ import print_function, division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from ggplot import *
import json

import os
from os.path import join
print('Current directory: {}'.format(os.getcwd()))
import sys
sys.path.append('..')
from glob import glob
from pprint import pprint
from time import time

DATA_PATH = '../../bucket/data/'
RESULTS_PATH = '../../bucket/results/'

Current directory: /Users/hmourit/Documents/0project/MScThesis/notebooks


In [2]:
result_files = glob(join(RESULTS_PATH, '*_*.json'))
result_files = [x for x in result_files if os.path.basename(x).startswith(('anova', 'infogain', 'rfe', 'chi2', 'mrmr'))]

In [3]:
def jaccard_distance(a, b):
    if not isinstance(a, set):
        a = set(a)
    if not isinstance(b, set):
        b = set(b)
    union_size = len(a | b)
    inter_size = len(a & b)
    return (union_size - inter_size) / union_size

In [5]:
result_files = glob(join(RESULTS_PATH, '*_*.json'))
result_files = [x for x in result_files if os.path.basename(x).startswith(('anova', 'infogain', 'rfe', 'chi2', 'mrmr'))]

from collections import defaultdict

clf = 'en'
filter_ = 'mrmr'

results = defaultdict(list)
for f in result_files:
    exp_id = f.split('_')[-1].rstrip('.json')
    try:
        d = json.load(open(join(RESULTS_PATH, f), 'r'))
    except ValueError as e:
        if os.path.getsize(f) == 0:
            e = 'File size is 0. Removing.'
            os.remove(f)
        print('{} -> {}'.format(f, e))
    base = {'exp_id': exp_id}
    left = []
    for k, v in d.items():
        if k != 'experiments':
            base[k] = v
    if os.path.basename(f).startswith('mrmr'):
        base['filter'] = 'mrmr'
    elif os.path.basename(f).startswith('rfe'):
        base['filter'] = 'rfe'
    
    try:
        if not (base['clf'] == clf and base['filter'] == filter_):
            continue
    except KeyError as e:
        print(e)
        print(f)
    for exp in d['experiments']:
        it = exp['iteration']
        if 'subsets' not in exp:
            print('{} -> Field "subsets" not found.'.format(f))
            continue
        for s in exp['subsets']:
#             train = accuracy_score(s['train']['y_true'], s['train']['y_pred'])
#             test = accuracy_score(s['test']['y_true'], s['test']['y_pred'])
            if 'n_features' in s:
                n_features = s['n_features']
            else:
                n_features = len(s['features'])   
#             results.append(dict(base, iteration=it, train=train, test=test, n_features=n_features))    
            key = (base['filter'], base['clf'], base['data'], base['tissue'], base['target'], n_features)
            results[key].append(set(s['features']))

'clf'
../../bucket/results/anova_-800824884543483809.json
../../bucket/results/anova_-800824884543483809.json -> Field "subsets" not found.
'clf'
../../bucket/results/anova_1773782541069684787.json
../../bucket/results/anova_1773782541069684787.json -> Field "subsets" not found.
'clf'
../../bucket/results/infogain_10_2347697712435705707.json
../../bucket/results/infogain_10_2347697712435705707.json -> Field "subsets" not found.
'clf'
../../bucket/results/infogain_exp_-905003234740235046.json
../../bucket/results/infogain_exp_-905003234740235046.json -> Field "subsets" not found.
'clf'
../../bucket/results/infogain_exp_7158626770039509150.json
../../bucket/results/infogain_exp_7158626770039509150.json -> Field "subsets" not found.


In [16]:
from itertools import combinations

consistency = {}
for x in results:
    n = len(results[x])
#     dist = 0.0
#     comb = 0
#     for i, j in combinations(xrange(n), 2):
#         dist += jaccard_distance(results[x][i], results[x][j])
#         comb += 1
#     dist /= comb
#     consistency[x] = dist
    inter = results[x][0]
    for i in xrange(1, n):
        inter &= results[x][i]
    consistency[x] = len(inter)

In [17]:
consistency

{('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 10): 0,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 20): 0,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 30): 0,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 40): 2,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 50): 3,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 60): 4,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 70): 5,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 80): 5,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 90): 5,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 100): 7,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 200): 13,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 300): 25,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u'ad.disease.status', 400): 32,
 ('mrmr', u'en', u'epi_ad', u'cerebellum', u

In [32]:
TABLES_FOLDER = '~/Dropbox/MSc Thesis/tables'
CAPTION_PLACEHOLDER = r'caption:placeholder'
import re

def my_replace(tex, name=None, src=None, dst=None, effects=None):
    name_map = {
        'en': (' en ', 'EN'),
        'svm': ('svm\_linear\_kernel', 'SVM'),
        'svm k': ('svm\_linear', 'SVM'),
        'l1 svm': ('svm\_linear\_l1', 'L1-SVM'),
        'clf': ('clf', 'CLF'),
        'pm': (r'\$\textbackslashpm\$', r'$\pm$'),
        'IG1': (r'infogain\_10', 'IG D1'),
        'IG2': (r'infogain\_exp', 'IG D2'),
        'anova': ('anova', 'ANOVA'),
        'mrmr': ('mrmr', 'mRMR'),
        'rfe': ('rfe', 'RFE'),
        'chi2': ('chi2', 'Chi2'),
        'stg': ('superior temporal gyrus', 'STG'),
        'wb': ('whole blood', 'WB'),
        'cer': ('cerebellum', 'CER'),
        'fc': ('frontal cortex', 'FC'),
        'ec': ('entorhinal cortex', 'EC')
    }
    
    effects_map = {
        '2': r'\mc{2}{c}{%s}',
        '2r': r'\multirow{2}{*}{%s}',
        '3r': r'\multirow{3}{*}{%s}',
        'b': r'\textbf{%s}',
        'c': r'\mc{1}{c}{%s}'
    }
    
    if name is not None:
        src = name_map[name][0]
        dst = name_map[name][1]
    
    if effects is not None:
        
        for effect in effects:
            dst = effects_map[effect] % dst
        
    return tex.replace(src, dst)  

def save_table(tex, filename, caption=None, folder=TABLES_FOLDER):
    folder = os.path.expanduser(folder)
    tex = tex.replace('tab:placeholder', 'tab:{}'.format(filename.split('.')[0]))
    if caption is not None:
        tex = tex.replace(CAPTION_PLACEHOLDER, caption)
    with open(os.path.join(folder, filename), 'w') as f:
        f.write(tex)
        
REMOVE_N_FEATURES = r'^n\\_features*'

def process_latex(tabular, remove_lines=None):
    if remove_lines and not isinstance(remove_lines, list):
        remove_lines = [remove_lines]
    
    purged_tabular = []
    for line in tabular.split('\n'):
        if r'\begin{tabular}' in line:
            begin_tabular = line.split('}{')
            line = begin_tabular[0] + '}'
            line += '{' + begin_tabular[1]#.replace('l', 'r')
            purged_tabular.append(line)
        elif remove_lines and not any(re.match(pattern, line) for pattern in remove_lines):
            purged_tabular.append(line)
    tabular = '\n'.join(purged_tabular)
        
    table_env_begin = r"""\begin{table}[!h]
                       \centering
                       \footnotesize
                       """.replace(' ', '')
    table_env_end = r"""\caption{caption:placeholder}
                     \label{tab:placeholder}
                     \end{table}
                     """.replace(' ', '')
    
    return table_env_begin + tabular + table_env_end

In [35]:
import os
FILE = '../../selection/probes_annotated.tsv'
import pandas as pd
foo = pd.read_csv(FILE, sep='\t', index_col=0)
foo = foo.iloc[:, :2]
foo_latex = foo.to_latex(longtable=True)
# with open('/Users/hmourit/Dropbox/MSc Thesis/tables/mdd_probes_annotated.tex', 'w') as f:
#     f.write(foo_latex)
foo_latex = process_latex(foo_latex, remove_lines=REMOVE_N_FEATURES)
foo_latex = my_replace(foo_latex, src='Gene Symbol', dst='Gene Symbol', effects='bc')
foo_latex = my_replace(foo_latex, src='Gene Title', dst='Gene Title', effects='bc')
foo_latex = my_replace(foo_latex, src='Probe Set ID', dst='Probe Set ID', effects='bc')
save_table(foo_latex, 'mdd_probes_annotated_long.tex', caption='16 most important probes related with stress condition in MDD dataset.')
foo

,Gene Symbol,Gene Title
Probe Set ID,,
1415899_at,Junb,jun B proto-oncogene
1415998_at,Vdac1,voltage-dependent anion channel 1
1416038_at,Snd1,staphylococcal nuclease and tudor domain conta...
1416147_at,Hspa4,heat shock protein 4
1416207_at,Taz,tafazzin
1416493_at,Ddost,dolichyl-di-phosphooligosaccharide-protein gly...
1416730_at,Rcl1,RNA terminal phosphate cyclase-like 1
1416937_at,Gabarap,gamma-aminobutyric acid receptor associated pr...
1417421_at,S100a1,S100 calcium binding protein A1
